# Tutorial: Counterfactual explanations for scorecard with binary target

Counterfactual explanations is one of the post-hoc methods used to provide explainability to
machine learning models. In this tutorial, we show how to generate optimal counterfactual explanations for scorecard models.

In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression

from optbinning import BinningProcess
from optbinning import Scorecard
from optbinning.scorecard import Counterfactual

We load the adult income dataset from the UCI repository (https://archive.ics.uci.edu/ml/datasets/adult). For this example, we select 8 features. The target "income" is transformed to binary (0: low-income, <=50K) and (1: high-income, >50K).

In [2]:
df = pd.read_csv("data/adult.data", sep=",", header=None)

columns = ["age", "workclass", "fnlwgt", "education", "education-num",
           "marital-status", "occupation", "relationship", "race", "sex",
           "capital-gain", "capital-loss", "hours-per-week","native-country",
           "income"]

target = "income"

variable_names = ["age", "workclass", "education", "marital-status",
                  "occupation", "race", "sex", "hours-per-week"]

df.columns = columns
X = df[variable_names]
y = (df[target].values == ' >50K').astype(int)

#### Scorecard model

First, we require a scorecard model. As shown in previous tutorials, we need a binning process and an estimator.

In [3]:
binning_process = BinningProcess(variable_names)

estimator = LogisticRegression(solver="lbfgs", class_weight="balanced")

scorecard = Scorecard(binning_process=binning_process,
                      estimator=estimator, scaling_method="min_max",
                      scaling_method_params={"min": 300, "max": 850})

scorecard.fit(X, y)

Scorecard(binning_process=BinningProcess(variable_names=['age', 'workclass',
                                                         'education',
                                                         'marital-status',
                                                         'occupation', 'race',
                                                         'sex',
                                                         'hours-per-week']),
          estimator=LogisticRegression(class_weight='balanced'),
          scaling_method='min_max',
          scaling_method_params={'max': 850, 'min': 300})

#### Generating counterfactual explanations - binary outcome

Having fitted the scorecard model, we can start generating counterfactual explanations.

##### Single counterfactual

As an input data point or query, we choose the sample with the lowest probability of having a high income.

In [4]:
idx_lowest = np.argmin(scorecard.predict_proba(df)[:, 1])
query = X.iloc[idx_lowest, :].to_dict()

In [5]:
query

{'age': 17,
 'workclass': ' Private',
 'education': ' 11th',
 'marital-status': ' Never-married',
 'occupation': ' Other-service',
 'race': ' Black',
 'sex': ' Female',
 'hours-per-week': 25}

A counterfactual class can simply be instantiated with a scorecard model. The first step is to fit the counterfactual with the data used to develop the scorecard. This allows computing problem data required for the integer programming formulation. This method must be called once before generating counterfactual explanations.

In [6]:
cf = Counterfactual(scorecard=scorecard)

In [7]:
cf.fit(X)

Counterfactual(scorecard=Scorecard(binning_process=BinningProcess(variable_names=['age',
                                                                                  'workclass',
                                                                                  'education',
                                                                                  'marital-status',
                                                                                  'occupation',
                                                                                  'race',
                                                                                  'sex',
                                                                                  'hours-per-week']),
                                   estimator=LogisticRegression(class_weight='balanced'),
                                   scaling_method='min_max',
                                   scaling_method_params={'max': 850,
                       

The minimum parameters required to generate counterfactuals are a ``query``, the desired target outcome ``y``, the outcome type, and the number of counterfactuals ``n_cf``. Other parameters such as the maximum number of changes ``max_changes`` can be specified.

In [8]:
cf.generate(query=query, y=1, outcome_type="binary", n_cf=1, max_changes=3)

Counterfactual(scorecard=Scorecard(binning_process=BinningProcess(variable_names=['age',
                                                                                  'workclass',
                                                                                  'education',
                                                                                  'marital-status',
                                                                                  'occupation',
                                                                                  'race',
                                                                                  'sex',
                                                                                  'hours-per-week']),
                                   estimator=LogisticRegression(class_weight='balanced'),
                                   scaling_method='min_max',
                                   scaling_method_params={'max': 850,
                       

The property ``status`` retrieves the status of the optimization solver. Additional information is retrieved by using the ``information`` method, showing three sections:

- Solver statistics: number of variables and constraints, and objective value.
- Objectives: value of the objectives functions, by default proximity and closeness.
- Timing: time statistics.

In [9]:
cf.status

'OPTIMAL'

In [10]:
cf.information(print_level=2)

optbinning (Version 0.18.0)
Copyright (c) 2019-2023 Guillermo Navas-Palencia, Apache License 2.0

  Begin options
    scorecard                            yes   * U
    special_missing                    False   * d
    n_jobs                                 1   * d
    verbose                            False   * d
  End options

  Status  : OPTIMAL                         

  Solver statistics
    Type                                 mip
    Number of variables                   42
    Number of constraints                 88
    Objective value                  16.1490
    Best objective bound             16.1490

  Objectives
    proximity                         3.7558
    closeness                        12.3932

  Timing
    Total time                          0.34 sec
    Fit                                 0.20 sec   ( 57.71%)
    Solver                              0.13 sec   ( 39.53%)
    Post-processing                     0.01 sec   (  6.98%)



The resulting optimal counterfactual explanation can be shown using ``display`` method. By default, all feature values are displayed. We might prefer to only observe the changes from the query, then set parameter ``show_only_changes`` to True. In addition, to add the counterfactual outcome, set parameter ``show_outcome`` to True.

In [11]:
cf.display()

,age,workclass,education,marital-status,occupation,race,sex,hours-per-week
0,"[33.50, 35.50)",Private,"[ Masters, Prof-school, Doctorate]","[ Married-AF-spouse, Married-civ-spouse]",Other-service,Black,Female,25


In [12]:
cf.display(show_only_changes=True)

,age,workclass,education,marital-status,occupation,race,sex,hours-per-week
0,"[33.50, 35.50)",-,"[ Masters, Prof-school, Doctorate]","[ Married-AF-spouse, Married-civ-spouse]",-,-,-,-


In [13]:
cf.display(show_only_changes=True, show_outcome=True)

,age,workclass,education,marital-status,occupation,race,sex,hours-per-week,outcome
0,"[33.50, 35.50)",-,"[ Masters, Prof-school, Doctorate]","[ Married-AF-spouse, Married-civ-spouse]",-,-,-,-,0.505995


##### Actionable features

Note that, in general, a scorecard includes features that are not mutable or that should not be actionable. For example, we might exclude marital status by choosing only those actionable features using the parameter ``actionable_features``.

In [14]:
cf.generate(query=query, y=1, outcome_type="binary", n_cf=1, max_changes=4,
            actionable_features=["age", "workclass", "education", "occupation",
                                 "hours-per-week"]
           ).display(show_only_changes=True, show_outcome=True)

,age,workclass,education,marital-status,occupation,race,sex,hours-per-week,outcome
0,"[43.50, 49.50)",-,"[ Masters, Prof-school, Doctorate]",-,"[ Tech-support, Protective-serv, Prof-specia...",-,-,"[39.50, 41.50)",0.503416


##### Weighted vs hierarchical method

Currently, there are two methods (weighted and hierarchical) supported to handle multiple objectives functions. By default, ``method`` is set to "weighted", due to having an inferior computational cost.

In [15]:
cf.generate(query=query, y=1, outcome_type="binary", n_cf=1,
            method="weighted", max_changes=4).display()

,age,workclass,education,marital-status,occupation,race,sex,hours-per-week
0,"[37.50, 40.50)",Private,[ Bachelors],"[ Married-AF-spouse, Married-civ-spouse]","[ ?, Armed-Forces, Farming-fishing]",Black,Female,25


In [16]:
cf.information()

optbinning (Version 0.18.0)
Copyright (c) 2019-2023 Guillermo Navas-Palencia, Apache License 2.0

  Status  : OPTIMAL                         

  Solver statistics
    Type                                 mip
    Number of variables                   42
    Number of constraints                 88
    Objective value                  14.0037
    Best objective bound             14.0024

  Objectives
    proximity                         4.1646
    closeness                         9.8391

  Timing
    Total time                          0.37 sec
    Fit                                 0.20 sec   ( 52.66%)
    Solver                              0.17 sec   ( 45.30%)
    Post-processing                     0.01 sec   (  4.52%)



When generating counterfactuals the objectives are passed as a dictionary. If ``method="weighted"``, the dictionary values represent the assigned weight to each objective, whereas if ``method="hierarchical"``, the dictionary values represent the priority of each objective. In the following example, we assign a higher weight to closeness, reducing the objective value compared to the previous one, where both weights are set to 1.

In [17]:
cf.generate(query=query, y=1, outcome_type="binary", n_cf=1,
            method="weighted", objectives={"proximity": 0.1, "closeness": 0.9},
            max_changes=4
           ).display()

,age,workclass,education,marital-status,occupation,race,sex,hours-per-week
0,"[43.50, 49.50)",Private,"[ Masters, Prof-school, Doctorate]",Never-married,"[ Tech-support, Protective-serv, Prof-specia...",Black,Female,"[39.50, 41.50)"


In [18]:
cf.information()

optbinning (Version 0.18.0)
Copyright (c) 2019-2023 Guillermo Navas-Palencia, Apache License 2.0

  Status  : OPTIMAL                         

  Solver statistics
    Type                                 mip
    Number of variables                   42
    Number of constraints                 88
    Objective value                   9.0395
    Best objective bound              9.0395

  Objectives
    proximity                         5.5872
    closeness                         9.4231

  Timing
    Total time                          0.47 sec
    Fit                                 0.20 sec   ( 41.28%)
    Solver                              0.27 sec   ( 56.82%)
    Post-processing                     0.01 sec   (  3.35%)



Considering the hierarchical method, the highest priority is assigned to proximity, therefore, we get the minimum objective value with a small degradation (``"priority_tol"=0.1``).

In [19]:
cf.generate(query=query, y=1, outcome_type="binary", n_cf=1,
            method="hierarchical", max_changes=4
           ).display()

,age,workclass,education,marital-status,occupation,race,sex,hours-per-week
0,"[35.50, 37.50)",Private,"[ Masters, Prof-school, Doctorate]","[ Married-AF-spouse, Married-civ-spouse]",Other-service,Black,Female,"[25.50, 34.50)"


In [20]:
cf.information()

optbinning (Version 0.18.0)
Copyright (c) 2019-2023 Guillermo Navas-Palencia, Apache License 2.0

  Status  : OPTIMAL                         

  Solver statistics
    Type                                 mip
    Number of variables                   44
    Number of constraints                 90
    Objective value                  12.0849
    Best objective bound             12.0849

  Objectives
    proximity                         3.9645
    closeness                        12.0849

  Timing
    Total time                          0.29 sec
    Fit                                 0.20 sec   ( 66.65%)
    Solver                              0.09 sec   ( 30.81%)
    Post-processing                     0.01 sec   (  8.22%)



On the contrary, if we switch priorities and reduce the priority degradation, we obtain the same closeness objective value as the one generated with the weight ``"closeness": 0.9``.

In [21]:
cf.generate(query=query, y=1, outcome_type="binary", n_cf=1,
            method="hierarchical", objectives={"proximity": 1, "closeness": 2},
            priority_tol=0.001,
            max_changes=4
           ).display()

,age,workclass,education,marital-status,occupation,race,sex,hours-per-week
0,"[43.50, 49.50)",Private,"[ Masters, Prof-school, Doctorate]",Never-married,"[ Tech-support, Protective-serv, Prof-specia...",Black,Female,"[39.50, 41.50)"


In [22]:
cf.information()

optbinning (Version 0.18.0)
Copyright (c) 2019-2023 Guillermo Navas-Palencia, Apache License 2.0

  Status  : OPTIMAL                         

  Solver statistics
    Type                                 mip
    Number of variables                   44
    Number of constraints                 90
    Objective value                   5.5872
    Best objective bound              5.5872

  Objectives
    closeness                         9.4231
    proximity                         5.5872

  Timing
    Total time                          1.02 sec
    Fit                                 0.20 sec   ( 19.23%)
    Solver                              0.82 sec   ( 80.09%)
    Post-processing                     0.01 sec   (  0.85%)



The hierarchical method becomes more interesting when several objectives come into play, as shown going forward.

##### Multiple counterfactuals

Now, let's focus on the generation of multiple counterfactual explanations for a single input data point. The present implementation is able to generate ``n_cf`` counterfactuals simultaneously while imposing diversity constraints.

For example, we generate three counterfactuals such that a maximum of four features can be changed and enforcing (hard constraint) a diversity of features. The diversity of features to change guarantees different combinations of changed features for each counterfactual.

In [23]:
cf.generate(query=query, y=1, outcome_type="binary", n_cf=3, max_changes=4,
            hard_constraints=["diversity_features"], time_limit=5
           ).display(show_only_changes=True, show_outcome=True)

,age,workclass,education,marital-status,occupation,race,sex,hours-per-week,outcome
0,"[49.50, 54.50)",-,"[ Masters, Prof-school, Doctorate]",-,[ Sales],-,-,"[41.50, 49.50)",0.529762
0,"[40.50, 43.50)",-,[ Bachelors],"[ Married-AF-spouse, Married-civ-spouse]",-,-,-,"[34.50, 39.50)",0.537689
0,"[37.50, 40.50)",-,[ Bachelors],"[ Married-AF-spouse, Married-civ-spouse]","[ ?, Armed-Forces, Farming-fishing]",-,-,-,0.508597


The generated counterfactuals change the following features:

- age | education | occupation | hours-per-week
- age | education | marital-status | hours-per-week
- age | education | marital-status | occupation

In [24]:
cf.information()

optbinning (Version 0.18.0)
Copyright (c) 2019-2023 Guillermo Navas-Palencia, Apache License 2.0

  Status  : FEASIBLE                        

  Solver statistics
    Type                                 mip
    Number of variables                  957
    Number of constraints                408
    Objective value                  44.0906
    Best objective bound             40.1499

  Objectives
    proximity                        14.1659
    closeness                        29.9247

  Timing
    Total time                          5.38 sec
    Fit                                 0.20 sec   (  3.64%)
    Solver                              5.17 sec   ( 96.00%)
    Post-processing                     0.02 sec   (  0.38%)



The reported status is only feasible, not optimal. When generating multiple counterfactual adding several constraints, the time required to prove optimality might be in the order of 10 - 120s, so one needs to increase ``time_limit``. However, good feasible solutions are generally achieved within a few seconds. Adding extra 5 seconds to ``time_limit``, the proximity and closeness objective values are reduced by 0.4% and 0.8%, respectively.

In [25]:
cf.generate(query=query, y=1, outcome_type="binary", n_cf=3, max_changes=4,
            hard_constraints=["diversity_features"], time_limit=15
           ).display(show_only_changes=True, show_outcome=True)

,age,workclass,education,marital-status,occupation,race,sex,hours-per-week,outcome
0,"[43.50, 49.50)",-,"[ Masters, Prof-school, Doctorate]",-,"[ Tech-support, Protective-serv, Prof-specia...",-,-,"[39.50, 41.50)",0.503416
0,"[40.50, 43.50)",-,[ Bachelors],"[ Married-AF-spouse, Married-civ-spouse]",-,-,-,"[34.50, 39.50)",0.537689
0,"[37.50, 40.50)",-,[ Bachelors],"[ Married-AF-spouse, Married-civ-spouse]","[ ?, Armed-Forces, Farming-fishing]",-,-,-,0.508597


In [26]:
cf.information()

optbinning (Version 0.18.0)
Copyright (c) 2019-2023 Guillermo Navas-Palencia, Apache License 2.0

  Status  : OPTIMAL                         

  Solver statistics
    Type                                 mip
    Number of variables                  957
    Number of constraints                408
    Objective value                  43.7051
    Best objective bound             43.7051

  Objectives
    proximity                        14.0688
    closeness                        29.6363

  Timing
    Total time                         10.84 sec
    Fit                                 0.20 sec   (  1.81%)
    Solver                             10.62 sec   ( 98.03%)
    Post-processing                     0.02 sec   (  0.16%)



The previous conterfactuals, although showing diversity regarding features to change, some feature values appear several times. To guarantee unique combinations of feature changes and feature values, we add the hard constraint "diversity_values".

In [27]:
cf.generate(query=query, y=1, outcome_type="binary", n_cf=3, max_changes=4,
            hard_constraints=["diversity_features", "diversity_values"], time_limit=15
           ).display(show_only_changes=True, show_outcome=True)

,age,workclass,education,marital-status,occupation,race,sex,hours-per-week,outcome
0,"[40.50, 43.50)",-,"[ Assoc-acdm, Assoc-voc]","[ Married-AF-spouse, Married-civ-spouse]",-,-,-,"[41.50, 49.50)",0.529430
0,"[49.50, 54.50)",-,"[ Masters, Prof-school, Doctorate]","[ Married-spouse-absent, Widowed, Divorced]","[ Tech-support, Protective-serv, Prof-specia...",-,-,-,0.510208
0,"[43.50, 49.50)",-,[ Bachelors],-,[ Exec-managerial],-,-,"[49.50, 55.50)",0.518267


In [28]:
cf.information()

optbinning (Version 0.18.0)
Copyright (c) 2019-2023 Guillermo Navas-Palencia, Apache License 2.0

  Status  : OPTIMAL                         

  Solver statistics
    Type                                 mip
    Number of variables                  981
    Number of constraints                408
    Objective value                  47.7067
    Best objective bound             47.7067

  Objectives
    proximity                        16.0514
    closeness                        31.6553

  Timing
    Total time                         14.17 sec
    Fit                                 0.20 sec   (  1.38%)
    Solver                             13.95 sec   ( 98.49%)
    Post-processing                     0.02 sec   (  0.13%)



Adding this extra constraint, the objective values slightly increase.

#### Generating counterfactual explanations - probability outcome

In some situations, having an outcome = 1 with a probability outcome > 0.5 might not be sufficient. For example, an algorithm might require a probability of 0.7 for acceptance.

In [29]:
df_query = pd.DataFrame([query],columns=query.keys())

In [30]:
scorecard.predict_proba(df_query)

array([[9.99831890e-01, 1.68109773e-04]])

Initially, the probability of the query is close to 0. When ``outcome_type="probability"``, at least an additional constraint must be provided. The supported constraints are:

- soft constraints: "diff_outcome".
- hard constraints: "min_outcome" and "max_outcome".

In this first try, we aim to obtain a counterfactual with an outcome probability as close as possible to 0.7. The weighted approach is used, assigning unit weights to all objectives.

In [31]:
cf.generate(query=df_query, y=0.7, outcome_type="probability", n_cf=2, max_changes=4,
            hard_constraints=["diversity_features"],
            soft_constraints={"diff_outcome": 1},
            ).display(show_only_changes=True, show_outcome=True)

,age,workclass,education,marital-status,occupation,race,sex,hours-per-week,outcome
0,"[23.50, 25.50)",-,[ HS-grad],-,"[ ?, Armed-Forces, Farming-fishing]",-,-,"[25.50, 34.50)",0.007926
0,"[23.50, 25.50)",-,[ HS-grad],-,"[ ?, Armed-Forces, Farming-fishing]",-,-,-,0.007168


A weight=1 is not large enough given the rest of the objective values. A workaround might be to increase the corresponding weight.

In [32]:
cf.generate(query=df_query, y=0.7, outcome_type="probability", n_cf=2, max_changes=4,
            hard_constraints=["diversity_features"],
            soft_constraints={"diff_outcome": 100}
           ).display(show_only_changes=True, show_outcome=True)

,age,workclass,education,marital-status,occupation,race,sex,hours-per-week,outcome
0,"[40.50, 43.50)",-,"[ Masters, Prof-school, Doctorate]","[ Married-AF-spouse, Married-civ-spouse]",-,-,-,"[34.50, 39.50)",0.697510
0,"[43.50, 49.50)",-,"[ Masters, Prof-school, Doctorate]","[ Married-AF-spouse, Married-civ-spouse]","[ ?, Armed-Forces, Farming-fishing]",-,-,-,0.702461


In [33]:
cf.information()

optbinning (Version 0.18.0)
Copyright (c) 2019-2023 Guillermo Navas-Palencia, Apache License 2.0

  Status  : OPTIMAL                         

  Solver statistics
    Type                                 mip
    Number of variables                  493
    Number of constraints                290
    Objective value                  32.4569
    Best objective bound             32.4569

  Objectives
    proximity                         9.1380
    closeness                        22.6536
    diff_outcome                      0.0067

  Timing
    Total time                          2.22 sec
    Fit                                 0.20 sec   (  8.82%)
    Solver                              2.01 sec   ( 90.57%)
    Post-processing                     0.01 sec   (  0.67%)



We might, however, want to guarantee a minimum probability outcome of 0.7. To do so, we add the hard constraint "min_outcome".

In [34]:
cf.generate(query=df_query, y=0.7, outcome_type="probability", n_cf=2, max_changes=4,
            hard_constraints=["diversity_features", "min_outcome"]
           ).display(show_only_changes=True, show_outcome=True)

,age,workclass,education,marital-status,occupation,race,sex,hours-per-week,outcome
0,"[49.50, 54.50)",-,"[ Masters, Prof-school, Doctorate]","[ Married-AF-spouse, Married-civ-spouse]",-,-,-,"[34.50, 39.50)",0.733405
0,"[49.50, 54.50)",-,"[ Masters, Prof-school, Doctorate]","[ Married-AF-spouse, Married-civ-spouse]","[ ?, Armed-Forces, Farming-fishing]",-,-,-,0.715094


In [35]:
cf.information()

optbinning (Version 0.18.0)
Copyright (c) 2019-2023 Guillermo Navas-Palencia, Apache License 2.0

  Status  : OPTIMAL                         

  Solver statistics
    Type                                 mip
    Number of variables                  493
    Number of constraints                290
    Objective value                  31.7672
    Best objective bound             31.7672

  Objectives
    proximity                         9.2342
    closeness                        22.5330

  Timing
    Total time                          1.55 sec
    Fit                                 0.20 sec   ( 12.64%)
    Solver                              1.34 sec   ( 86.53%)
    Post-processing                     0.01 sec   (  0.96%)



Moreover, it is possible to combine the previous constraints with diversity constraints.

In [36]:
cf.generate(query=df_query, y=0.7, outcome_type="probability", n_cf=2,
            max_changes=4, method="hierarchical",
            objectives={"proximity": 2, "closeness": 1},
            hard_constraints=["min_outcome"],
            soft_constraints={"diversity_features": 2, "diversity_values": 1}
           ).display(show_only_changes=True, show_outcome=True)

,age,workclass,education,marital-status,occupation,race,sex,hours-per-week,outcome
0,"[49.50, 54.50)",-,"[ Masters, Prof-school, Doctorate]","[ Married-AF-spouse, Married-civ-spouse]","[ ?, Armed-Forces, Farming-fishing]",-,-,-,0.715094
0,"[40.50, 43.50)",-,[ Bachelors],"[ Married-AF-spouse, Married-civ-spouse]",-,-,-,"[49.50, 55.50)",0.718292


In [37]:
cf.information()

optbinning (Version 0.18.0)
Copyright (c) 2019-2023 Guillermo Navas-Palencia, Apache License 2.0

  Status  : OPTIMAL                         

  Solver statistics
    Type                                 mip
    Number of variables                  498
    Number of constraints                296
    Objective value                  -6.0000
    Best objective bound             -6.0000

  Objectives
    proximity                         9.9447
    diversity_features                2.0000
    closeness                        23.7715
    diversity_values                  6.0000

  Timing
    Total time                          3.11 sec
    Fit                                 0.20 sec   (  6.30%)
    Solver                              2.90 sec   ( 93.29%)
    Post-processing                     0.01 sec   (  0.44%)



#### Special and missing bins

The scorecard models might have bins for special and missing values. The parameter ``special_missing`` permits including these bins as feasible solutions for the generation of counterfactual explanations.

In [38]:
cf = Counterfactual(scorecard=scorecard, special_missing=True)

In [39]:
cf.fit(X)

Counterfactual(scorecard=Scorecard(binning_process=BinningProcess(variable_names=['age',
                                                                                  'workclass',
                                                                                  'education',
                                                                                  'marital-status',
                                                                                  'occupation',
                                                                                  'race',
                                                                                  'sex',
                                                                                  'hours-per-week']),
                                   estimator=LogisticRegression(class_weight='balanced'),
                                   scaling_method='min_max',
                                   scaling_method_params={'max': 850,
                       

In [40]:
cf.generate(query=query, y=1, outcome_type="binary", n_cf=1, max_changes=4
           ).display(show_only_changes=True)

,age,workclass,education,marital-status,occupation,race,sex,hours-per-week
0,"[49.50, 54.50)",-,"[ Assoc-acdm, Assoc-voc]","[ Married-AF-spouse, Married-civ-spouse]",-,-,-,Missing
